<a href="https://colab.research.google.com/github/NiclasFenton-Wiegleb/schlager-lyrics-bot/blob/main/Schlage_bot_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install transformers==4.3
!pip install datasets
# !pip install peft
# !pip install accelerate
# !pip install bitsandbytes
# !pip install trl
# !pip install safetensors
# !pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 34.9 MB/s eta 0:00:00


In [2]:
#Check CUDA is running on GPU and bf16 supported (required)

import torch
print(torch.cuda.is_available(), torch.cuda.is_bf16_supported())

True False


In [3]:
#Import dependencies

from random import randrange
import pandas as pd
from datasets import Dataset

In [4]:
df_lyrics = pd.read_csv('schlager_songs_v2.csv')

df_lyrics['lyrics'][10]

'{"status"false,"reason""Unexpected error occurred (no quota cost) Please try again later"}'

In [5]:
def convert(string):
    li = list(string.split(" "))
    return li

In [6]:
df_lyrics.head(15)

,Unnamed: 0.1,Unnamed: 0,url,title,spotify_id,release_year,artist_name,lyrics,verse_1
0,0,0,https://open.spotify.com/track/6KBwcS9yjfUa1RX...,Schwarze Natascha,6KBwcS9yjfUa1RXtSXfER6,2015-03-13,Peter Wackel,Hey\n Hey\n Hey\n (Hey) In einer Kneipe bei B...,Hey Hey Hey (Hey) In einer Kneipe bei Bier un...
1,1,1,https://open.spotify.com/track/5euK1jyFJOV3Zy8...,Die Krüge hoch!,5euK1jyFJOV3Zy80Ihowun,2015-03-13,Buddy,"Jetzt geht es los, los, los\n Die Krüge hoch,...","Jetzt geht es los, los, los"
2,2,2,https://open.spotify.com/track/4puMtZzXGNWAT6S...,Die schönste Frau der Welt,4puMtZzXGNWAT6SRoywSWm,2015-06-19,Markus Becker,"Du bist die schönste, schönste, schönste Frau...","Du bist die schönste, schönste, schönste Frau..."
3,3,3,https://open.spotify.com/track/4G01BR6tZms9zI9...,Mallorca wir sind wieder da - Video Edit,4G01BR6tZms9zI9IIoiEqi,2014-06-13,Stefan Stürmer,Wie jedes Jahr\n Malle wir sind wieder da\n O...,Wie jedes Jahr Malle wir sind wieder da Ole O...
4,4,4,https://open.spotify.com/track/3VWsSRR9qg9tw1a...,"Polka, Polka, Polka - Club Mix",3VWsSRR9qg9tw1a8SNGf0s,2015-02-27,Brings,Der Wind is kalt und rau\n Und mer wisse janz...,Der Wind is kalt und rau
5,5,5,https://open.spotify.com/track/5WsBHZRPeXLmZTa...,Kölsche Jung - Party Edit,5WsBHZRPeXLmZTaIOZbB1e,2013-01-01,Brings,Oh oh oh yeoh\n Oh oh oh yeoh\n Oh oh oh yeoh...,Oh oh oh yeoh Oh oh oh yeoh Oh oh oh yeoh Oh ...
6,6,6,https://open.spotify.com/track/4jUOYnaifqgXeen...,"Biste braun, kriegste Fraun",4jUOYnaifqgXeenKBiR062,2014-01-01,Mickie Krause,Ich hab Muskeln und 'nen durchtrainierten Bau...,Ich hab Muskeln und 'nen durchtrainierten Bauch
7,7,7,https://open.spotify.com/track/5ryh7FtwZBm6cjP...,Wir sind wieder da - L'amour toujours Version,5ryh7FtwZBm6cjPYFTwqLx,2015-05-10,Andy Bar,Wir sind wieder da\n Wie jedes Jahr\n Hier is...,Wir sind wieder da Wie jedes Jahr Hier ist An...
8,8,8,https://open.spotify.com/track/1xBqk0cath1OTSD...,Oh Baby - Originaltitel: Heaven Is a Place On ...,1xBqk0cath1OTSDJDHzzzA,2014-01-10,Mia Julia,"Oh Baby, komm doch her zu mir\n Es ist mir eg...","Oh Baby, komm doch her zu mir"
9,9,9,https://open.spotify.com/track/0h1sVP9YeZJdTeN...,Mallorca (Da bin ich daheim),0h1sVP9YeZJdTeNFoozhle,2015-04-24,DJ Mico,Du bist der geilste Ort der Welt\n Bist unser...,Du bist der geilste Ort der Welt


In [7]:
output=[]

input=[]

for ind in df_lyrics["lyrics"].index:
  lyrics = df_lyrics["lyrics"].iloc[ind]
  verse_1 = df_lyrics["verse_1"].iloc[ind]
  if verse_1 == None:
    continue
  else:
    try:
      #Input
      verse_list = convert(verse_1)
      cleaned_list = [x for x in verse_list if x != '']
      input.append(cleaned_list)

      #Output
      lyrics_list = convert(lyrics)
      cleaned_lyrics = [x for x in lyrics_list if x != '']
      output.append(cleaned_lyrics)
    except:
      continue

df_dataset = pd.DataFrame(columns=['input', 'output'])

df_dataset['input'] = input
df_dataset['output'] = output

df_dataset.head(15)

,input,output
0,"[Hey, Hey, Hey, (Hey), In, einer, Kneipe, bei,...","[Hey\n, Hey\n, Hey\n, (Hey), In, einer, Kneipe..."
1,"[Jetzt, geht, es, los,, los,, los]","[Jetzt, geht, es, los,, los,, los\n, Die, Krüg..."
2,"[Du, bist, die, schönste,, schönste,, schönste...","[Du, bist, die, schönste,, schönste,, schönste..."
3,"[Wie, jedes, Jahr, Malle, wir, sind, wieder, d...","[Wie, jedes, Jahr\n, Malle, wir, sind, wieder,..."
4,"[Der, Wind, is, kalt, und, rau]","[Der, Wind, is, kalt, und, rau\n, Und, mer, wi..."
5,"[Oh, oh, oh, yeoh, Oh, oh, oh, yeoh, Oh, oh, o...","[Oh, oh, oh, yeoh\n, Oh, oh, oh, yeoh\n, Oh, o..."
6,"[Ich, hab, Muskeln, und, 'nen, durchtrainierte...","[Ich, hab, Muskeln, und, 'nen, durchtrainierte..."
7,"[Wir, sind, wieder, da, Wie, jedes, Jahr, Hier...","[Wir, sind, wieder, da\n, Wie, jedes, Jahr\n, ..."
8,"[Oh, Baby,, komm, doch, her, zu, mir]","[Oh, Baby,, komm, doch, her, zu, mir\n, Es, is..."
9,"[Du, bist, der, geilste, Ort, der, Welt]","[Du, bist, der, geilste, Ort, der, Welt\n, Bis..."


In [ ]:
''' DON'T RUN'''
# Create separate column for 1st verse

output=[]

input=[]

for ind in df_lyrics["lyrics"].index:
    lyrics = df_lyrics["lyrics"].iloc[ind]
    if lyrics == '{"status"false,"reason""Unexpected error occurred (no quota cost) Please try again later"}' :
        input.append(None)
    else:
        try:
            verse = lyrics.split("\n")[0]
            n = 1
            while len(' '.join(verse_1).split()) <= 15:
                verse_1 = lyrics.split("\n")[0:n]
                print(verse_1)
                n += 1
            # convert(verse_1)
            # clean_verse = [x for x in verse_1 if str(x) != 'nan']
            input.append(''.join(verse_1))
            output.append(convert(lyrics))

        except:
          continue
            # verse_list.append(None)


# df_lyrics["verse_1"] = verse_list

# list = convert(df_lyrics["verse_1"][0])
# cleanedList = [x for x in list if str(x) != 'nan']

# cleanedList

input

# df_lyrics.to_csv("schlager_songs_v2.csv")

In [6]:
# Prompt Example

### Instruction:

"Benuzte den unten gegebenen Vers um den Text für ein Lied zu schreiben."

### Input:

"Du sagst nicht ein Wort"

### Response:

f""" Du sagst nicht ein Wort
 Und deine Hand wischt eine Träne fort
 Und dein leerer Blick
 Sinkt in dein Glas
 Du sitzt hier vor mir
 Und dein Gesicht lässt keinen Zweifel mehr
 Heut sagst du mir
 Dass ich dich verlier'
 Nie war Zeit für dich
 Ich lebte nur in meiner eignen Welt
 Ich weiß, du wirst gehen
 Ich muss dich versteh'n
 Lieb mich ein letztes mal
 Es bleibt mir keine andre Wahl
 Ich weiß, dass ich die Nacht mit dir
 An den Tag verlier'
...
 Und bleib bei mir"""

" Du sagst nicht ein Wort\n Und deine Hand wischt eine Träne fort\n Und dein leerer Blick\n Sinkt in dein Glas\n Du sitzt hier vor mir\n Und dein Gesicht lässt keinen Zweifel mehr\n Heut sagst du mir\n Dass ich dich verlier'\n Nie war Zeit für dich\n Ich lebte nur in meiner eignen Welt\n Ich weiß, du wirst gehen\n Ich muss dich versteh'n\n Lieb mich ein letztes mal\n Es bleibt mir keine andre Wahl\n Ich weiß, dass ich die Nacht mit dir\n An den Tag verlier'\n...\n Und bleib bei mir"

In [8]:
def format_instruction(sample):

    output_texts = []

    for i in range(len(sample['input'])):
      text = f"""### Anweisung:
Benuzte den unten gegebenen Vers um den Text für ein Lied zu schreiben.

### Input:

{' '.join(sample['input'])}

### Output:

{' '.join(sample['output'])}"""

      output_texts.append(text)

    return output_texts


In [8]:
''' DON'T RUN'''
#Create input and output dataset from dataframe

df_dataset = df_lyrics[["verse_1", "lyrics"]]

ind_nan = []

for ind in df_dataset.index:
    if df_dataset["verse_1"].iloc[ind] == None:
        ind_nan.append(ind)
    else:
        continue

df_dataset.drop(index=ind_nan, inplace=True)

df_dataset.reset_index(drop=True, inplace=True)

df_dataset.rename(columns={"lyrics" : "output", "verse_1" : "input"}, inplace= True)

<ipython-input-8-933d3ca1faf6>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dataset.drop(index=ind_nan, inplace=True)
<ipython-input-8-933d3ca1faf6>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dataset.rename(columns={"lyrics" : "output", "verse_1" : "input"}, inplace= True)


In [9]:
print(len(df_dataset))

1046


In [10]:
train_df = df_dataset[:946]
test_df = df_dataset[946:]

In [11]:
from datasets.dataset_dict import DatasetDict

datasets_train_test = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "test": Dataset.from_pandas(test_df)
    })

datasets_train_test

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 946
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 100
    })
})

In [12]:
print(format_instruction(datasets_train_test["train"][randrange(946)])[0])

### Anweisung:
Benuzte den unten gegebenen Vers um den Text für ein Lied zu schreiben.

### Input:

Come on, come on, come on, come on

### Output:

Come on, come on, come on, come on
 (Together) We will go our way
 (Together) We will leave someday
 (Together) Your hand in my hand
 (Together) We will make our plans
 (Together) We will fly so high
 (Together) Tell all our friends good-bye
 (Together) We will start life new
 (Together) This is what we'll do
 (Go West) Life is peaceful there
 (Go West) In the open air
 (Go West) Where the skies are blue
 (Go West) This is what we're gonna do
 (Go West, this is what we're gonna do, Go West)
 (Together) We will love the beach
 (Together) We will learn and teach
 (Together) Change our pace of life
 (Together) We will work and strive
 (I love you) I know you love me
 (I want you) How could I disagree?
 (So that's why) I make no protest
 (When you say) You will do the rest
 (Go West) Life is peaceful there
 (Go West) In the open air
 (Go West)

*Break Start*

In [ ]:
# Import dependencies

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, default_data_collator, get_linear_schedule_with_warmup
from trl import SFTTrainer
import accelerate
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
#SFTTrainer supports a native integration with peft, which makes it easy to instruction tune LLMs

from peft import get_peft_config, get_peft_model, PromptTuningInit, PromptTuningConfig, TaskType, PeftType


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
#Parameters
device = "cuda"
model_id = "malteos/bloom-1b5-clp-german" # non-gated

peft_config = PromptTuningConfig(
            task_type="CAUSAL_LM",
            prompt_tuning_init=PromptTuningInit.TEXT,
            num_virtual_tokens=8,
            prompt_tuning_init_text="Benuzte den gegebenen Input um ein Schlager Lied zu schreiben.",
            tokenizer_name_or_path=model_id
)

dataset_name = "schlager_lyrics"
checkpoint_name = f"{dataset_name}_{model_id}_{peft_config.peft_type}_{peft_config.task_type}_v1.pt".replace(
    "/", "_"
)
text_column = "input"
label_column = "output"
max_length = 300 #optimize
lr = 6e-2
num_epochs = 30
batch_size = 3


Preprocess dataset

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
def preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{x}" for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.pad_token_id]
        # print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocess_function(datasets_train_test["train"][0:10])

In [ ]:
processed_datasets = datasets_train_test.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=datasets_train_test["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/946 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["test"]


train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=batch_size,
    pin_memory=True
)
eval_dataloader = DataLoader(
    eval_dataset,
    collate_fn=default_data_collator,
    batch_size=batch_size,
    pin_memory=True)

Train

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id)
model = get_peft_model(model, peft_config)
print(model.print_trainable_parameters())

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 32,768 || all params: 6,738,448,384 || trainable%: 0.0004862840543203603
None


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [ ]:
#Maximising the space available for model loading on GPU
os.environ['PYTORCH_CUDA_ALLOC_CONF']="max_split_size_mb:1024"


In [ ]:
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True))

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

OutOfMemoryError: ignored

Notes:

v01:

- Epoch 21: Loss is levelling out
- Training Loss of last epoch (50) = 2.1387

Inference Test: 5.5/10

v02:

- optimise GPU memory allocation to make use of unallocated space
- parameters:
  - max_length = 500
  - lr = 2e-4
  - num_epochs = 50
  - batch_size = 5

- Training Loss of last epoch (50) = 1.5997
- Evaluation Loss of last epoch (50) = 1.5925

Inference Test: 4/10

v03:

- changed prompt to: "Benuzte den gegebenen Input um ein Schlager Lied zu schreiben." (previously: "Benuzte den gegebenen Vers um den Text fur ein Schlager Lied zu schreiben."

- parameters:
  - max_length = 300
  - lr = 3e-2
  - num_epochs = 30
  - batch_size = 6

- Training Loss of last epoch (30) = 1.8726
- Evaluation Loss of last epoch (30) = 1.9066

Inference Test: 6.5/10

v04

- parameters:
  - max_length = 500
  - lr = 6e-2
  - num_epochs = 30
  - batch_size = 5

- Training Loss of last epoch (30) = 1.5883
- Evaluation Loss of last epoch (30) = 1.5758

Inference Test: 7.5/10

v05

- change base model to:
malteos/bloom-6b4-clp-german

- parameters:
  - max_length = 500
  - lr = 8e-2
  - num_epochs = 30
  - batch_size = 5

- Training Loss of last epoch (30) =
- Evaluation Loss of last epoch (30) =

Inference Test: /10

*Share model to huggingface hub*

In [ ]:
peft_model_id = "niclasfw/bloom-1b5-schlager-bot-004"
model.push_to_hub(peft_model_id, use_auth_token=True)

NameError: ignored

*Inference*

In [32]:
from peft import PeftModel, PeftConfig

peft_model_id = "niclasfw/bloom-1b5-schlager-bot-004"
device = "cuda"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)


(…)bot-004/resolve/main/adapter_config.json:   0%|          | 0.00/500 [00:00<?, ?B/s]

KeyboardInterrupt: ignored

In [ ]:
model = model.to(device)
sample = f"""Wir feiern das Leben laut\n
Wir feiern das Leben laut\n
Wir stellen keine Fragen, das Leben wird uns tragen\n
"""

prompt = f"""### Instruction:
          Benuzte den gegebenen Input um ein Schlager Lied zu schreiben.

### Input:
{sample}

"""

for n in range(5):
        input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
        # with torch.inference_mode():
        outputs = model.generate(input_ids=input_ids, pad_token_id=tokenizer.eos_token_id, max_new_tokens=300, do_sample=True, top_p=0.9,temperature=0.9)

        print(f"Version {n+1}")
        print(f"Prompt:\n{sample}\n")
        print(f"Generated output:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")

OutOfMemoryError: ignored

*Training 7b model*

In [13]:
!pip install -q bitsandbytes==0.41.1 transformers==4.33.3 accelerate==0.23.0 datasets==2.14.5 einops==0.6.1
!pip install -q -U git+https://github.com/huggingface/peft.git@69665f24e98dc5f20a430637a31f196158b6e0da
# !pip install flash-attn --no-build-isolation
!pip install trl

In [14]:
import os
import re
import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import  load_dataset, Dataset, Value
from peft import (
    LoraConfig,
    PeftConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

In [16]:
from huggingface_hub import notebook_login

notebook_login()

In [35]:
#Parameters
device = "cuda"
model_id = "LeoLM/leo-hessianai-7b" # non-gated
max_length = 300 #optimize
lr = 6e-2
num_epochs = 5
batch_size = 8
text_column = "input"
label_column = "output"
output_dir = "./content/drive/MyDrive/AiCore/trained_model/schlager-bot-001"

In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=False,
    quantization_config=bnb_config,
)

model = prepare_model_for_kbit_training(model)

tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)
tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right"

(…)eo-hessianai-7b/resolve/main/config.json:   0%|          | 0.00/708 [00:00<?, ?B/s]

(…)esolve/main/pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

(…)i-7b/resolve/main/generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

(…)ai-7b/resolve/main/tokenizer_config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

(…)hessianai-7b/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)-7b/resolve/main/special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [25]:
def preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{x}" for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.pad_token_id]
        # print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocess_function(datasets_train_test["train"][0:10])

In [27]:
processed_datasets = datasets_train_test.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=datasets_train_test["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/946 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

In [28]:
train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["test"]

In [19]:
# LoRA config based on QLoRA paper
config = LoraConfig(
    r=16,
    lora_alpha=32,
    # target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, config)

In [36]:
from transformers import TrainingArguments

# training_args = TrainingArguments(
#     output_dir=output_dir,
#     num_train_epochs=num_epochs,
#     per_device_train_batch_size=batch_size,
#     gradient_accumulation_steps=2,
#     gradient_checkpointing=True,
#     optim="paged_adamw_32bit",
#     logging_steps=10,
#     save_strategy="epoch",
#     learning_rate=2e-4,
#     # bf16=True,
#     fp16=True,
#     # tf32=True,
#     max_grad_norm=0.3,
#     warmup_ratio=0.03,
#     lr_scheduler_type="constant",
#     disable_tqdm=True,# disable tqdm since with packing values are in correct
# )

training_args = TrainingArguments(
    # evaluation_strategy='epoch',
    # auto_find_batch_size=True, # Try to auto-find a batch size.
    # Also see https://huggingface.co/google/flan-ul2/discussions/16#64c8bdaf4cc48498134a0271
    learning_rate=lr,
    # bf16=True, # Only on A100
    fp16=True, # On V100
    save_total_limit=4,
    # warmup_steps=2,
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    output_dir='checkpoints',
    # save_strategy='epoch',
    report_to="none",
    logging_steps=25, # Number of steps between logs.
    save_safetensors=True,
    # load_best_model_at_end=True,
    metric_for_best_model='accuracy',
)

In [21]:
#Maximising the space available for model loading on GPU
os.environ['PYTORCH_CUDA_ALLOC_CONF']="max_split_size_mb:1024"

In [22]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=datasets_train_test['train'],
    peft_config=config,
    max_seq_length=max_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format_instruction,
    args=training_args,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:548: UserWarning: The passed formatting_func has more than one argument. Usually that function should have a single argument `example` which corresponds to the dictionary returned by each element of the dataset. Make sure you know what you are doing.
  warnings.warn(


In [37]:
# train
trainer.train() # there will not be a progress bar since tqdm is disabled

AttributeError: ignored

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=processed_datasets["train"],
    # eval_dataset=processed_datasets["test"], # 16GB GPU not big enough
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    # compute_metrics=compute_metrics,
)
model.config.use_cache = False
trainer.train(resume_from_checkpoint=False) # Set to true if resuming
trainer.save_model("final_model")

Step,Training Loss
25,0.000000


7B model:

v01:

- parameters:
  - lr = 6e-2
  - num_epochs = 5
  - batch_size = auto


In [ ]:
peft_model_id = "niclasfw/leo-hessianai-7b-schlager-bot-001"
model.push_to_hub(peft_model_id, use_auth_token=True)

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model=peft_model_id)

In [ ]:
outputs = generator("Wo war ich in der Nacht von Freitag auf Montag? War ich drei, drei Tage wach oder einfach im Koma?",
                     num_return_sequences=1, max_length=500)
outputs

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: ignored

In [ ]:
#encode context the generation is conditioned on
model_inputs = tokenizer('I enjoy walking with my cute dog', return_tensors='pt').to(torch_device)

# generate 40 new tokens
greedy_output = model.generate(**model_inputs, max_new_tokens=40)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

In [ ]:
#Test input
inputs = tokenizer(
    f'{input} : {"Wo war ich in der Nacht von Freitag auf Montag? War ich drei, drei Tage wach oder einfach im Koma?"} output :',
    return_tensors="pt",
)

In [ ]:
model.to(device)

with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(
        inputs=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=10, eos_token_id=3
    )
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


['<bound method Kernel.raw_input of <google.colab._kernel.Kernel object at 0x78d097e5e2c0>> : Wo war ich in der Nacht von Freitag auf Montag? War ich drei, drei Tage wach oder einfach im Koma? output : <google.colab._kernel.']


*Break End*

In [ ]:
# Hugging Face model id
model_id = "malteos/bloom-6b4-clp-german" # non-gated


# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


bloom_model = BloomModelForCausalLM.from_pretrained(
      model_id,
      load_in_8bit=True,
      # quantization_config=bnb_config,
      use_cache=False,
      torch_dtype=torch.float16,
      device_map="auto")

# model.config.pretraining_tp = 1


Loading checkpoint shards:   0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
#Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right"

In [ ]:
# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM"
)


# prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [ ]:
#We need to define the hyperparameters before we can start training

args = TrainingArguments(
    output_dir="./content/drive/MyDrive/AiCore/trained_model/schlager-bot-001",
    num_train_epochs=3,
    per_device_train_batch_size= 4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    # bf16=True,
    # tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    # disable_tqdm=True  # disable tqdm since with packing values are in correct
)

In [ ]:
#We can now configure the trainer

max_seq_length = 2048 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    # peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format_instruction,
    args=args,
)

Map:   0%|          | 0/1046 [00:00<?, ? examples/s]

In [ ]:
# train
trainer.train() # there will not be a progress bar since tqdm is disabled

# save model
trainer.save_model()